<a href="https://colab.research.google.com/github/ctctmusic/SARMIA-Sales-Forecasting-Gradio-Interface-/blob/main/Final_Project_Sales_Forecasting_SARIMA_Gradio_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#1.Install Dependencies
!pip install gradio statsmodels openpyxl

In [5]:
#2. Upload Excel File
from google.colab import files
uploaded = files.upload()


Saving US_Regional_Sales_Data.xlsx to US_Regional_Sales_Data.xlsx


In [6]:
#3. load and prepare your data
import pandas as pd

# Load the uploaded Excel file
file_name = list(uploaded.keys())[0]
sales_df = pd.read_excel(file_name, sheet_name='Sales Orders Sheet')

# Clean + calculate net revenue
sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'])
sales_df['Net Revenue'] = sales_df['Order Quantity'] * sales_df['Unit Price'] * (1 - sales_df['Discount Applied'])

# Aggregate to quarterly revenue by sales channel
quarterly_df = (
    sales_df
    .groupby([pd.Grouper(key='OrderDate', freq='Q'), 'Sales Channel'])['Net Revenue']
    .sum()
    .reset_index()
)


<ipython-input-6-1eda7387c4be>:15: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  .groupby([pd.Grouper(key='OrderDate', freq='Q'), 'Sales Channel'])['Net Revenue']


In [7]:
#4. Define SARIMA forecast function
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

def sarima_forecast_plot(channel: str, periods: int):
    ts = quarterly_df[quarterly_df['Sales Channel'] == channel].set_index('OrderDate')['Net Revenue']
    model = SARIMAX(ts, order=(1,1,1), seasonal_order=(1,1,1,4), enforce_stationarity=False, enforce_invertibility=False)
    results = model.fit(disp=False)
    forecast = results.get_forecast(steps=periods)
    forecast_index = pd.date_range(start=ts.index[-1] + pd.offsets.QuarterEnd(), periods=periods, freq='Q')
    forecast_series = pd.Series(forecast.predicted_mean.values, index=forecast_index)

    fig, ax = plt.subplots(figsize=(12, 6))
    ts.plot(ax=ax, label='Actual')
    forecast_series.plot(ax=ax, style='--', label='Forecast')
    ax.set_title(f'SARIMA Forecast for {channel} Sales ({periods} Quarters Ahead)')
    ax.set_xlabel('Quarter')
    ax.set_ylabel('Net Revenue (USD)')
    ax.grid(True)
    ax.legend()
    plt.tight_layout()
    return fig


In [8]:
#5. Launch the Gradio App in Colab
import gradio as gr

channel_list = quarterly_df['Sales Channel'].unique().tolist()

demo = gr.Interface(
    fn=sarima_forecast_plot,
    inputs=[
        gr.Dropdown(choices=channel_list, label="Sales Channel"),
        gr.Slider(minimum=1, maximum=12, value=4, step=1, label="Forecast Horizon (Quarters)")
    ],
    outputs=gr.Plot(label="SARIMA Forecast"),
    title="Quarterly Sales Forecast (SARIMA Model)",
    description="Select a sales channel and forecast the next quarters using SARIMA."
)

demo.launch(share=True)  # Use share=True to get a public link



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80b2b6b9d9d7ea6ea8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
